# Battle of the Neighbourhoods - Toronto
### Author: Shubham Mor

In [2]:
import pandas as pd

# Importing Data

In [3]:
import requests
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_page = requests.get(url)
wiki_page

<Response [200]>

# Now we import beautifulSoup and read the html contents

In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup (wiki_page.content, features="lxml")

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df

PostalCode                Borough  \
0          M3A             North York   
1          M4A             North York   
2          M5A       Downtown Toronto   
3          M6A             North York   
4          M7A           Queen's Park   
..         ...                    ...   
98         M8X              Etobicoke   
99         M4Y       Downtown Toronto   
100        M7Y  East Toronto Business   
101        M8Y              Etobicoke   
102        M8Z              Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Regent Park, Harbourfront  
3                     Lawrence Manor, Lawrence Heights  
4                        Ontario Provincial Government  
..                                                 ...  
98       The Kingsway, Montgomery Road, Old Mill North  
99                                Church and Wellesley  
100                                     Enclave of M4L  
101  Old Mill South, King's Mill Park, Sunnylea, Hu...  
102  Mimico NW, The Queensway West, South of Bloor,...  

[103 rows x 3 columns]

In [6]:
df.shape

(103, 3)

# Since Geocoder was returning None values consistently, we have used the csv

In [8]:
import os, types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_ce515c52776e46e88187382d68844781 = 'https://s3.ap.cloud-object-storage.appdomain.cloud'
else:
    endpoint_ce515c52776e46e88187382d68844781 = 'https://s3.private.ap.cloud-object-storage.appdomain.cloud'

client_ce515c52776e46e88187382d68844781 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='yCYvGgWSa9TCKI_pa9ZGXJ79C-KKOM5hq849xkEsJmq5',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_ce515c52776e46e88187382d68844781)

body = client_ce515c52776e46e88187382d68844781.get_object(Bucket='courseraworkspace-donotdelete-pr-mh6kotypb3us1j',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

latlongdf = pd.read_csv(body)
latlongdf.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [14]:
latitudes = []
longitudes = []
for postal_code in df['PostalCode']:
    print(latlongdf[latlongdf['Postal Code']==postal_code]['Latitude'])
    latitudes.append(latlongdf[latlongdf['Postal Code']==postal_code]['Latitude'])
    longitudes.append(latlongdf[latlongdf['Postal Code']==postal_code]['Longitude'])
# print(longitudes)
df['Latitude'] = latitudes
df['Longitude'] = longitudes

25    43.753259
Name: Latitude, dtype: float64
34    43.725882
Name: Latitude, dtype: float64
53    43.65426
Name: Latitude, dtype: float64
71    43.718518
Name: Latitude, dtype: float64
85    43.662301
Name: Latitude, dtype: float64
93    43.667856
Name: Latitude, dtype: float64
0    43.806686
Name: Latitude, dtype: float64
26    43.745906
Name: Latitude, dtype: float64
35    43.706397
Name: Latitude, dtype: float64
54    43.657162
Name: Latitude, dtype: float64
72    43.709577
Name: Latitude, dtype: float64
94    43.650943
Name: Latitude, dtype: float64
1    43.784535
Name: Latitude, dtype: float64
27    43.7259
Name: Latitude, dtype: float64
36    43.695344
Name: Latitude, dtype: float64
55    43.651494
Name: Latitude, dtype: float64
73    43.693781
Name: Latitude, dtype: float64
95    43.643515
Name: Latitude, dtype: float64
2    43.763573
Name: Latitude, dtype: float64
37    43.676357
Name: Latitude, dtype: float64
56    43.644771
Name: Latitude, dtype: float64
74    43.689026
Nam

In [13]:
df


PostalCode                Borough  \
0          M3A             North York   
1          M4A             North York   
2          M5A       Downtown Toronto   
3          M6A             North York   
4          M7A           Queen's Park   
..         ...                    ...   
98         M8X              Etobicoke   
99         M4Y       Downtown Toronto   
100        M7Y  East Toronto Business   
101        M8Y              Etobicoke   
102        M8Z              Etobicoke   

                                          Neighborhood  \
0                                            Parkwoods   
1                                     Victoria Village   
2                            Regent Park, Harbourfront   
3                     Lawrence Manor, Lawrence Heights   
4                        Ontario Provincial Government   
..                                                 ...   
98       The Kingsway, Montgomery Road, Old Mill North   
99                                Church and Wellesley   
100                                     Enclave of M4L   
101  Old Mill South, King's Mill Park, Sunnylea, Hu...   
102  Mimico NW, The Queensway West, South of Bloor,...   

                                           Latitude  \
0    25    43.753259
Name: Latitude, dtype: float64   
1    34    43.725882
Name: Latitude, dtype: float64   
2     53    43.65426
Name: Latitude, dtype: float64   
3    71    43.718518
Name: Latitude, dtype: float64   
4    85    43.662301
Name: Latitude, dtype: float64   
..                                              ...   
98   90    43.653654
Name: Latitude, dtype: float64   
99    52    43.66586
Name: Latitude, dtype: float64   
100  87    43.662744
Name: Latitude, dtype: float64   
101  91    43.636258
Name: Latitude, dtype: float64   
102  92    43.628841
Name: Latitude, dtype: float64   

                                           Longitude  
0    25   -79.329656
Name: Longitude, dtype: float64  
1    34   -79.315572
Name: Longitude, dtype: float64  
2    53   -79.360636
Name: Longitude, dtype: float64  
3    71   -79.464763
Name: Longitude, dtype: float64  
4    85   -79.389494
Name: Longitude, dtype: float64  
..                                               ...  
98   90   -79.506944
Name: Longitude, dtype: float64  
99    52   -79.38316
Name: Longitude, dtype: float64  
100  87   -79.321558
Name: Longitude, dtype: float64  
101  91   -79.498509
Name: Longitude, dtype: float64  
102  92   -79.520999
Name: Longitude, dtype: float64  

[103 rows x 5 columns]